# Linear Optimization (CS5040) Assignment 4

## Authors

| Name | Roll Number |
|-|-|
| Gautam Singh | CS21BTECH11018 |
| Varun Gupta | CS21BTECH11060 |
| Anshul Sangrame | CS21BTECH11004 |

## Setup

In [8]:
# Install libraries
%pip install numpy

# Import libraries
import numpy as np

Note: you may need to restart the kernel to use updated packages.


In [9]:
# Parameters to run the program go here
INPUT_FILE = '../data/input_start.csv'    # Input file path
DELIMITER = ','                     # Delimiter in input file
EPS = 1e-6                          # Threshold for perturbation

## Input Handling

In [10]:
def handle_input(fname: str, delimiter: str=',') -> (np.ndarray, np.ndarray, np.ndarray, np.ndarray):
    """
    Handle input from CSV file.
    """
    # Take input from CSV file into numpy array
    input_arr = np.genfromtxt(fname, delimiter=delimiter, skip_header=0)

    # Values of A, b, c ,X
    A = input_arr[2:, :-1]
    b = input_arr[2:, -1]
    c = input_arr[1, :-1]
    X = input_arr[0,:-1]

    # Check for bad inputs, and exit if found
    if np.any(np.isnan(A)):
        raise IOError('Matrix A contains bad input:', A)
    if np.any(np.isnan(b)):
        raise IOError('Matrix b contains bad input:', b)
    if np.any(np.isnan(c)):
        raise IOError('Matrix c contains bad input:', c)
    if np.any(np.isnan(X)):
        raise IOError('Matrix z contains bad input:', X)

    # Values of m and n
    m, n = A.shape
    # Check if A is full rank
    if np.linalg.matrix_rank(A) != n:
        raise np.linalg.LinAlgError('Matrix A is not full rank:', A)

    return A,b,c,X

## Handling Degeneracy

In [11]:
def isdegenerate(
    A: np.ndarray,
    b: np.ndarray,
    c: np.ndarray,
    X_check: np.ndarray
) -> bool:
    """
    Method to check if the given linear programming problem is degenerate.        
    """
    m, n = A.shape

    # If number of tight rows is not equal to n, then the problem is degenerate.
    if np.isclose(A@X_check, b).sum() == n:
        return False
    return True

def perturb(
    A: np.ndarray, 
    b: np.ndarray, 
    c: np.ndarray,
    X_check: np.ndarray,
    eps: float=1e-2,
    n_iter: int=1000,
) -> (np.ndarray, np.ndarray, np.ndarray):
    """
    Method to remove degeneracy by perturbing the last m-n rows of b by a
    small value if needed.
    """
    # Do not perturb if nondegenerate
    if not isdegenerate(A, b, c,X_check):
        return A, b, c
    # Find dimensions of A
    m, n = A.shape
    # Initialize a random number generator
    rng = np.random.default_rng()
    # Perturb for certain number of iterations
    while n_iter:
        _b = b
        # Choose a small value to add at random
        eps_rng = rng.uniform(eps, 10*eps)
        b[n:] += eps_rng**np.arange(1,m-n+1)
        # Return if not degenerate
        if not isdegenerate(A, b, c,X_check):
            return A, _b, c
        n_iter -= 1
    # Raise error if perturbation fails
    raise RuntimeError('Could not perturb the linear programming problem')


## Finding the Optimal Vertex

In [12]:
def vertex_directions(
    A: np.ndarray,
    b: np.ndarray,
    c: np.ndarray,
    v: np.ndarray,
) -> np.ndarray:
    """
    Function to find directions of the other vertices of the polytope from given
    vertex.
    """
    tight_rows = np.where(np.isclose(A@v, b))
    A1 = A[tight_rows]
    return -np.linalg.inv(A1.T)

def simplex_neighbour(
    A: np.ndarray,
    b: np.ndarray,
    c: np.ndarray,
    u: np.ndarray,
) -> np.ndarray or None:
    """
    Function to find a neighbouring vertex with greater cost, or report that
    there is no such neighbour.
    """
    # Find directions to other vertices
    z = vertex_directions(A, b, c, u)

    # Find costs for each direction
    costs = z@c

    # Find directions which give positive cost
    costs_positive = np.where(costs > 0)[0]
    
    # If there are no such directions, declare optimality
    if len(costs_positive) == 0:
        return None
    else:
        # Get any direction with positive cost
        v = z[costs_positive[0]]

        # Check for unboundedness. If A@v keeps decreasing in that direction,
        # then the LP is unbounded.
        if len(np.where(A@v > 0)[0]) == 0:
            raise np.linalg.LinAlgError('LP is unbounded.')

        # Find untight rows
        untight_rows = np.where(~np.isclose(A@u, b))
        A2 = A[untight_rows]
        b2 = b[untight_rows]
        print(u, A2, b2)

        # Find feasible neighbour and required coefficients
        # Coefficients are (b2 - A2@u)/(A2@v)
        alpha = (b2-A2@u)/(A2@v)
        t = np.min(alpha[alpha >= 0])
        return u + t*v

def simplex(
    A: np.ndarray,
    b: np.ndarray,
    c: np.ndarray,
    u: np.ndarray,
    n_iter: int=10,
) -> np.ndarray:
    while n_iter:
        # Display vertex and cost
        print(u, c.T@u)
        # Find neighbour of a greater cost
        u1 = simplex_neighbour(A, b, c, u)
        if u1 is None:
            return u
        else:
            u = u1
        n_iter -= 1

## Driver 

In [13]:
if __name__ == "__main__":
    A, b, c, X = handle_input(INPUT_FILE, DELIMITER)
    A, b, c = perturb(A, b, c,X)
    simplex(A, b, c, X)

[0. 0.] 0.0
[0. 0.] [[1. 1.]] [1.]
[1. 0.] 2.0
[1. 0.] [[-1.  0.]] [0.]
[0. 1.] 3.0
